In [19]:
# Копируем папку
import shutil

dirname = "new_task"
src_dir = "./sample"
dst_dir = "./" + dirname
shutil.copytree(src_dir, dst_dir)


'./new_task'

In [20]:
task_name = "class_data_3.1.1"

solution_text = """
def make_set(element):
    new_set = set(element)
    return new_set, len(new_set)"""


legend = r"Для первого знакомства с множествами решите простую задачу. Напишите функцию \verb|make_set|, которая получает на вход одну строку или список чисел. Преобразуйте их в множество и посчитайте его мощность (количество элементов). На выходе функция должна отдавать кортеж из получившегося множества и его мощности."
input_info = r"Любой объект из следующих: \verb|list[Any]|, \verb|tuple[Any]|, \verb|str|, \verb|range|"
output_info = r"\verb|tuple[set[Any], int]|"
additional_notes = (
    r"Ваше решение должно содержать только функцию с именем \verb|make_set|, без её вызова и без ввода/вывода данных."
)

test_cases = [
    "тестовая_строка",
    "[4, 8, 15, 16, 23, 42, 15]",
    "[1, 2, 3, 4, 5]",
    "12345",
    "[1, 2, 2, 3, 4, 4, 5]",
    "[]",
    "''",
    "abc!@#123",
    "こんにちは",
    "['', ' ', 'a', ' ', '']",
    "['123', 123]",
    "[1.1, 1.10, 1.100]",
    "['🙂', '😀', '🙂']",
    "range(10000)",
    "['', None, False, 0, 0]",
]


In [21]:
# Поменять ./new_task/solutions/main.py


with open(f"./{dirname}/solutions/main.py", mode="w", encoding="utf-8") as file:
    file.write(solution_text)


In [22]:
# Сгенерить тесты


# TODO: LLM-кой сделать тесты
def create_test_cases() -> list[str]:
    pass


for i in range(len(test_cases)):
    with open(f"./{dirname}/tests/{i + 1:02}", mode="w", encoding="utf-8") as file_test:
        file_test.write(test_cases[i])


In [23]:
type_of_task = "function"  # "basic"


In [24]:
# Сгенерить ответы
import subprocess
import os

# К этому моменту в папке ответов еще нет, так что это количество именно тестов
n_tests = len([x for x in os.listdir(f"./{dirname}/tests/") if not x.endswith(".a")])


if type_of_task == "function":
    with open(f"./{dirname}/files/participantSolution.py", mode="w", encoding="utf-8") as file:
        file.write(solution_text)

    commands = [
        f"python ./{dirname}/files/run_tests.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]

    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)

    subprocess.run(f"del ./{dirname}/files/participantSolution.py", shell=True, text=True, capture_output=True)
else:
    # basic prog task
    commands = [
        f"python ./{dirname}/solutions/main.py < ./{dirname}/tests/{test_number:02} > ./{dirname}/tests/{test_number:02}.a"
        for test_number in range(1, n_tests + 1)
    ]
    for cmd in commands:
        process = subprocess.run(cmd, shell=True, text=True, capture_output=True)


In [25]:
# Копирование примера
import shutil

src_test = f"./{dirname}/tests/01"
src_ans = f"./{dirname}/tests/01.a"
dst_test = f"./{dirname}/statements/russian/example.01"
dst_ans = f"./{dirname}/statements/russian/example.01.a"
shutil.copy(src_test, dst_test)
shutil.copy(src_ans, dst_ans)


'./new_task/statements/russian/example.01.a'

In [26]:
# Изменяем problem.xml
with open(f"./{dirname}/problem.xml", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_task_name", task_name)
content = content.replace("placeholder_ntests", str(n_tests))

task_price = 1
if task_price > 1:
    content = content.replace("checker_1.py", f"checker_{task_price}.py")
    content = content.replace("files/checker_1.py", f"checker_{task_price}.py")

with open(f"./{dirname}/problem.xml", mode="w", encoding="utf-8") as file:
    file.write(content)


In [27]:
# Поменять условие
with open(f"./{dirname}/statements/russian/problem.tex", mode="r", encoding="utf-8") as file:
    content = file.read()


content = content.replace("placeholder_legend", legend)
content = content.replace("placeholder_input", input_info)
content = content.replace("placeholder_output", output_info)
content = content.replace("placeholder_note", additional_notes)

with open(f"./{dirname}/statements/russian/problem.tex", mode="w", encoding="utf-8") as file:
    file.write(content)
